In [80]:
from bs4 import BeautifulSoup
import urllib.request
import ssl
import pandas as pd
import os

def GetKospi200():     
    stockDic = dict()
    lastPageNum = 0

    # 마지막 페이지 찾기
    base_url = "http://finance.naver.com/sise/entryJongmok.nhn?&page="
    target_url = base_url + str(1)
    context = ssl._create_unverified_context()
    # soup = BeautifulSoup(urllib.request.urlopen(target_url).read(), "lxml")
    soup = BeautifulSoup(urllib.request.urlopen(target_url, context=context).read().decode('euc-kr', 'ignore'), "lxml")
    for item in soup.find_all('td'):
        if item.has_attr('class') and 'pgRR' in item['class']:
            lastPageNum = int(str(item.a['href']).replace('/sise/entryJongmok.nhn?&page=', ''))

    for i in range(1, lastPageNum+1):
        target_url = base_url + str(i)
        # soup = BeautifulSoup(urllib.request.urlopen(target_url).read(), "lxml")
        soup = BeautifulSoup(urllib.request.urlopen(target_url, context=context).read().decode('euc-kr', 'ignore'), "lxml")
        postNoList = soup.find_all('a')


        # 종목코드와 종목명 담기
        for item in postNoList:
            if item.has_attr('target') and '_parent' in item['target'] and item.has_attr('href'):
                if str(item['href']).startswith('/item/main.nhn?code='):
                    stockDic[str(item['href']).replace('/item/main.nhn?code=', '')] = item.text
    return stockDic

def getKospiIndex():
    outFileName = "data/kospi.csv"
    i = 1
    last_date = ""
    while True:
        if os.path.exists(outFileName):
            df = pd.read_csv(outFileName, names=["date", "index", "diff", "diff_ratio", "amount", "tot_price"])

        with open(outFileName, mode = 'a') as f: 
            url = "https://finance.naver.com/sise/sise_index_day.nhn?code=KOSPI&page=" + str(i)
            print(url)            
            table = pd.read_html(url)
            table = table[0].dropna()           
            if last_date == table["날짜"].max():
                f.close()                
                break
            else:
                last_date = table["날짜"].max()
    #             table.to_csv(f, header=False, index=False)
                if len(table[~table["날짜"].isin(df["date"].tolist())]) == 0:
                    print("종료")
                    break
                table[~table["날짜"].isin(df["date"].tolist())].to_csv(f, header=False, index=False)
                i += 1

In [146]:
import pandas as pd

def GetPriceData(item):    
    
    # print(item)
    code = item[0]
    name = item[1]
                
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    url = "http://finance.naver.com/item/sise_day.nhn?code=" + code
    page = "&page="
    idx = 1

    outputFileName = directory + code + '.csv'
    
    df = pd.read_csv(outputFileName)
    bFlag = True
    with open(outputFileName, 'a') as f:
        while bFlag == True:
            fullAddr = url + page + str(idx)

            source_code = requests.get(fullAddr, headers = headers)
            if source_code is None:
                bFlag = False
                return

            soup = BeautifulSoup(source_code.text,"lxml")
            if soup.find('td', class_='on').find('a').text != str(idx):
                bFlag = False
                return

            for tr in filter(lambda x:x.get("onmouseout") is not None, soup.find_all("tr")):
                if tr.find("span",class_ = "tah p10 gray03") is None:
                    # 가격데이터가 없으면 False로 빠져나옴
                    bFlag = False
                    break

                else:
                    tDate = tr.find("span",class_ = "tah p10 gray03").text
                    cPrice = tr.find_all("span",class_ = "tah p11")
                    sIdx = 1

                    if len(cPrice) != 5 :
                        sIdx = 2

                    dt = tDate.replace("." ,"-")
                    if (df["date"] == dt).sum() > 0:                        
#                         print(dt, " 가격존재 ")
                        f.close()
                        bFlag = False
                        break

                    pClose = float(cPrice[0].text.replace("," ,""))
                    pStart = float(cPrice[sIdx].text.replace("," ,""))
                    sIdx += 1
                    pMax   = float(cPrice[sIdx].text.replace("," ,""))
                    sIdx += 1
                    pMin   = float(cPrice[sIdx].text.replace("," ,""))
                    sIdx += 1
                    amount = float(cPrice[sIdx].text.replace("," ,""))

                    stockList = []
                    stockList.append(dt)
                    stockList.append(code)
                    stockList.append(pStart)
                    stockList.append(pClose)
                    stockList.append(pMin)
                    stockList.append(pMax)
                    stockList.append(amount)                
                    f.writelines(",".join([str(x) for x in stockList]) + '\n') 
            idx += 1
    f.close()
    

In [138]:
from bs4 import BeautifulSoup
import urllib.request
import requests 
import os

def getFinanceInfo(code=None):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    # 투자정보
    # totalCnt = []  # 상장주식수
    # forignerHaveLimit = []  # 외국인한도보유주식수
    # forignerHaveCnt = []  # 외국인보유주식수
    # max52week = []  # 52주 최고
    # min52week = []  # 52주 최저
    # per = []
    # eps = []
    # per_eps_date = []
    # estimate_per = []  # 추정 PER
    # estimate_eps = []  # 추정 EPS
    # pbr = []
    # bps = []
    # pbr_bps_date = []
    # dvr = []  # 배당수익
    financeFileName = 'data/stock_finance_data.csv'
    cols = ["date",
             "code", 
             "totalCnt", 
             "forignerHaveLimit",
             "forignerHaveCnt",
             "min52week",
             "max52week",
             "per",
             "eps",
             "per_eps_date",
             "estimate_per",
             "estimate_eps",
             "pbr",
             "bps",
             "pbr_bps_date",
             "dvr"]
    if os.path.exists(financeFileName):
        df = pd.read_csv(financeFileName, header=None, names=cols, dtype=str)

    url = "https://finance.naver.com/item/main.nhn?code=" + code
    source_code = requests.get(url, headers = headers)
    if source_code is None:
        pass
    else:
        soup = BeautifulSoup(source_code.text, "lxml")
        date = soup.find(id='time').find_all('em')[0].text
        
        # 이미 존재하는 데이터면 skip
        if ((df["date"] == date) & (df["code"] == code)).sum() > 0:
            return
        
        totalCnt = soup.find(id='tab_con1').find_all('em')[2].text.replace(',', '').replace('N/A', '0')
        forignerHaveLimit = soup.find(id='tab_con1').find_all('em')[5].text.replace(',', '').replace('N/A', '0')
        forignerHaveCnt = soup.find(id='tab_con1').find_all('em')[6].text.replace(',', '').replace('N/A', '0')
        max52week = soup.find(id='tab_con1').find_all('em')[10].text.replace(',', '').replace('N/A', '0')
        min52week = soup.find(id='tab_con1').find_all('em')[11].text.replace(',', '').replace('N/A', '0')
        per = soup.find(id='tab_con1').find_all('em')[12].text.replace(',', '').replace('N/A', '0')
        eps = soup.find(id='tab_con1').find_all('em')[13].text.replace(',', '').replace('N/A', '0')
        per_eps_date = soup.find(id='tab_con1').find_all('span', class_='date')[0].text.replace('(','').replace(')','') if len(soup.find(id='tab_con1').find_all('span', class_='date')) > 0 else ""
        estimate_per = soup.find(id='tab_con1').find_all('em')[14].text.replace(',', '').replace('N/A', '0')
        estimate_eps = soup.find(id='tab_con1').find_all('em')[15].text.replace(',', '').replace('N/A', '0')
        pbr = soup.find(id='tab_con1').find_all('em')[16].text.replace(',', '').replace('N/A', '0')
        bps = soup.find(id='tab_con1').find_all('em')[17].text.replace(',', '').replace('N/A', '0')
        pbr_bps_date = soup.find(id='tab_con1').find_all('span', class_='date')[1].text.replace('(','').replace(')','') if len(soup.find(id='tab_con1').find_all('span', class_='date')) > 1 else ""
        dvr = soup.find(id='tab_con1').find_all('em')[18].text.replace(',', '').replace('N/A', '0')                
        
        financeList = []
        financeList.append(date)
        financeList.append(str(code))
        financeList.append(totalCnt)
        financeList.append(forignerHaveLimit)
        financeList.append(forignerHaveCnt)        
        financeList.append(max52week)
        financeList.append(min52week)
        financeList.append(per)
        financeList.append(eps)
        financeList.append(per_eps_date)
        financeList.append(estimate_per)
        financeList.append(estimate_eps)
        financeList.append(pbr)
        financeList.append(bps)
        financeList.append(pbr_bps_date)
        financeList.append(dvr)    
        
        with open(financeFileName, 'a') as f:
            f.writelines(",".join([str(x) for x in financeList]) + '\n') 
        f.close()

        

In [148]:
import os
import requests 
from datetime import datetime
import pandas as pd

def crawling():
#     file_list = os.listdir('data')
#     exist_list = []
#     for file in file_list:
#         if 'finance' not in file and '.csv' in file:
#             exist_list.append(file)

    directory = 'data/'
    stockDict = GetKospi200()
    # stockDict = {'192400' : '쿠쿠홀딩스'}

    kospiListFile = "kospi_list.csv"
    bFlag = False
    if os.path.exists(directory+kospiListFile):
        bFlag = bool((pd.read_csv(directory+kospiListFile, 
                                  names=["date", "code", "name"], 
                                  header=None)["date"] == datetime.today().strftime("%Y-%m-%d")).sum())
    
    with open(directory + "/" + kospiListFile, 'a') as f:
        for key in stockDict.keys():
            print(stockDict[key], key)
            if not bFlag:        
                l = []
                l.append(datetime.today().strftime("%Y-%m-%d"))
                l.append(key)
                l.append(stockDict[key])
                f.writelines(",".join([str(x) for x in l]) + '\n') 

            #if str(key) not in exist_list:
            target = (key, stockDict[key])  
            GetPriceData(target) 
            getFinanceInfo(key)
    f.close()

In [147]:
crawling()

삼성전자 005930
GetPriceData
getFinanceInfo
SK하이닉스 000660
GetPriceData
getFinanceInfo
NAVER 035420
GetPriceData
getFinanceInfo
LG화학 051910
GetPriceData
getFinanceInfo
현대차 005380
GetPriceData
getFinanceInfo
삼성바이오로직스 207940
GetPriceData
getFinanceInfo
삼성SDI 006400
GetPriceData
getFinanceInfo
카카오 035720
GetPriceData
getFinanceInfo
셀트리온 068270
GetPriceData
getFinanceInfo
기아차 000270
GetPriceData
getFinanceInfo
현대모비스 012330
GetPriceData
getFinanceInfo
POSCO 005490
GetPriceData
getFinanceInfo
SK이노베이션 096770
GetPriceData
getFinanceInfo
LG전자 066570
GetPriceData
getFinanceInfo
LG생활건강 051900
GetPriceData
getFinanceInfo
삼성물산 028260
GetPriceData
getFinanceInfo
엔씨소프트 036570
GetPriceData
getFinanceInfo
SK텔레콤 017670
GetPriceData
getFinanceInfo
SK 034730
GetPriceData
getFinanceInfo
KB금융 105560
GetPriceData


FileNotFoundError: [Errno 2] No such file or directory: 'data/105560.csv'

In [ ]:
# 특정날짜 데이터 제거 및 reset_index
import pandas as pd
def removeData(date = "2021-02-26"):
    directory = 'data'
    file_list = os.listdir(directory)

    for file in file_list:
        if 'finance' not in file and '.csv' in file and 'kospi' not in file:   
            print(file)
            df = pd.read_csv(directory + "/" + file)
            if "Unnamed: 0" in df.columns:
                df = pd.read_csv(directory + "/" + file).drop("Unnamed: 0", axis=1)

            if (df["date"] == date).sum() > 0:
                df = df.drop(df[df["date"] == date].index)
                df.to_csv(directory + "/" + file, index=False)

In [ ]:
# 특정날짜 데이터 제거 및 reset_index
import pandas as pd
def resetIndex():
#     cols = ["date", "code", "start", "close", "min", "max", "amount"]
    directory = 'data'
    file_list = os.listdir(directory)

    for file in file_list:
        if 'finance' not in file and '.csv' in file and 'kospi' not in file:     
            print(file)
            df = pd.read_csv(directory + "/" + file)
            if "Unnamed: 0" in df.columns:
                df = pd.read_csv(directory + "/" + file).drop("Unnamed: 0", axis=1)

            df = df.sort_values(by="date").reset_index().drop("index", axis=1)
            df.to_csv(directory + "/" + file, index=False)
                